<a href="https://colab.research.google.com/github/sh-0620/dacon-recommender-system/blob/main/%EC%9B%B9_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

python 라이브러리

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# NLTK 리소스 다운로드 (처음 한 번만 실행)
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, RSLPStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('rslp')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download pt_core_news_sm

In [6]:
import spacy
# 영어 및 포르투갈어 모델 로드

nlp_en = spacy.load('en_core_web_sm')
nlp_pt = spacy.load('pt_core_news_sm')

CSV 파일 로드

In [8]:
# CSV 파일 로드
data_path = '/content/drive/MyDrive/웹 기사 추천시스템/'

view_log_df = pd.read_csv(data_path + "view_log.csv")  # 유저 로그 데이터
articles_info_df = pd.read_csv(data_path +  "article_info.csv")  # 기사 데이터

영어(preprocess_text_en) 와 포르투칼(preprocess_text_pt) 을 나눠서 텍스트 전처리

In [9]:
#  영어 텍스트 전처리

def preprocess_text_en(text):
    # 소문자 변환
    text = text.lower()

    # 특수 문자 제거
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 토큰화
    tokens = word_tokenize(text)

    # 불용어 제거
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # 표제어 추출
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # 토큰들을 하나의 문자열로 결합
    processed_text = ' '.join(tokens)

    # 스페이스를 사용하여 명사 추출
    doc = nlp_en(processed_text)
    nouns = [token.text for token in doc if token.pos_ == 'NOUN']

    return nouns

#  포르투칼 텍스트 전처리

def preprocess_text_pt(text):
    # 소문자 변환
    text = text.lower()

    # 특수 문자 제거
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 토큰화
    tokens = word_tokenize(text)

    # 불용어 제거
    stop_words = set(stopwords.words('portuguese'))
    tokens = [word for word in tokens if word not in stop_words]

    # 어간 추출
    stemmer = RSLPStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # 토큰들을 하나의 문자열로 결합
    processed_text = ' '.join(tokens)

    # 스페이스를 사용하여 명사 추출
    doc = nlp_pt(processed_text)
    nouns = [token.text for token in doc if token.pos_ == 'NOUN']

    return nouns

# 언어별 전처리 함수 적용

In [10]:
# 언어별 전처리 함수 적용
articles_info_df['Processed_Title'] = articles_info_df.apply(lambda row: preprocess_text_pt(row['Title']) if row['Language'] == 'pt' else preprocess_text_en(row['Title']), axis=1)
articles_info_df['Processed_Content'] = articles_info_df.apply(lambda row: preprocess_text_pt(row['Content']) if row['Language'] == 'pt' else preprocess_text_en(row['Content']), axis=1)

In [11]:
articles_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   articleID          3008 non-null   object
 1   Title              3008 non-null   object
 2   Content            3008 non-null   object
 3   Format             3008 non-null   object
 4   Language           3008 non-null   object
 5   userID             3008 non-null   object
 6   userCountry        659 non-null    object
 7   userRegion         657 non-null    object
 8   Processed_Title    3008 non-null   object
 9   Processed_Content  3008 non-null   object
dtypes: object(10)
memory usage: 235.1+ KB


articles_info_df 에서 절반 이상의 데이터가 Null값인 'userCountry', 'userRegion' 제거 및 필요 없다고 예측되는 userID 컬럼 제거

In [12]:
# 불필요한 열 제거
articles_info_df = articles_info_df.drop(['userCountry', 'userRegion', 'userID'], axis=1)

articles_info_df 에서 view_log_df에 기록이 되지 않은 기사는 유의미 하지 않으므로 제거

In [13]:
# 데이터 병합
merged_df = pd.merge(view_log_df, articles_info_df, on='articleID')

# 기사 데이터 중 로그 데이터에 없는 기사 제거
articles_info_df = articles_info_df[articles_info_df['articleID'].isin(merged_df['articleID'].unique())]

콘텐츠 기반 필터링 + 협업 필터링 작업 할 때 shape을 맞추기 위해 데이터 merge 과정을 진행

In [14]:
# 데이터 병합
merged_df = pd.merge(view_log_df, articles_info_df, on='articleID')

In [15]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42717 entries, 0 to 42716
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   userID             42717 non-null  object
 1   articleID          42717 non-null  object
 2   userRegion         42717 non-null  object
 3   userCountry        42717 non-null  object
 4   Title              42717 non-null  object
 5   Content            42717 non-null  object
 6   Format             42717 non-null  object
 7   Language           42717 non-null  object
 8   Processed_Title    42717 non-null  object
 9   Processed_Content  42717 non-null  object
dtypes: object(10)
memory usage: 3.3+ MB


merge한 데이터 프레임 csv 파일로 저장

In [16]:
merged_df.to_csv(data_path + 'article_nouns_df.csv')